In [1]:
import pandas as pd
import numpy as np
import keras
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras import Sequential
import csv
import math

Using TensorFlow backend.


# read data

In [2]:
train_data = pd.read_csv("train.csv", encoding="big5")
print(train_data)

              日期  測站          測項     0     1     2     3     4     5     6  \
0       2014/1/1  豐原    AMB_TEMP    14    14    14    13    12    12    12   
1       2014/1/1  豐原         CH4   1.8   1.8   1.8   1.8   1.8   1.8   1.8   
2       2014/1/1  豐原          CO  0.51  0.41  0.39  0.37  0.35   0.3  0.37   
3       2014/1/1  豐原        NMHC   0.2  0.15  0.13  0.12  0.11  0.06   0.1   
4       2014/1/1  豐原          NO   0.9   0.6   0.5   1.7   1.8   1.5   1.9   
...          ...  ..         ...   ...   ...   ...   ...   ...   ...   ...   
4315  2014/12/20  豐原         THC   1.8   1.8   1.8   1.8   1.8   1.7   1.7   
4316  2014/12/20  豐原       WD_HR    46    13    61    44    55    68    66   
4317  2014/12/20  豐原  WIND_DIREC    36    55    72   327    74    52    59   
4318  2014/12/20  豐原  WIND_SPEED   1.9   2.4   1.9   2.8   2.3   1.9   2.1   
4319  2014/12/20  豐原       WS_HR   0.7   0.8   1.8     1   1.9   1.7   2.1   

      ...    14    15    16    17    18    19    20    21    22

In [3]:
test_data = pd.read_csv("test.csv",header=None)

r=range(test_data.shape[1]-2)
columns=['id','測向']
for i in r:
    columns.extend(str(i))
test_data.columns=columns
print(test_data)

          id          測向     0     1     2     3     4     5     6     7     8
0       id_0    AMB_TEMP    21    21    20    20    19    19    19    18    17
1       id_0         CH4   1.7   1.7   1.7   1.7   1.7   1.7   1.7   1.7   1.8
2       id_0          CO  0.39  0.36  0.36   0.4  0.53  0.55  0.34  0.31  0.23
3       id_0        NMHC  0.16  0.24  0.22  0.27  0.27  0.26  0.27  0.29   0.1
4       id_0          NO   1.3   1.3   1.3   1.3   1.4   1.6   1.2   1.1   0.9
...      ...         ...   ...   ...   ...   ...   ...   ...   ...   ...   ...
4315  id_239         THC   1.8   1.8   1.8   1.8   1.7   1.7   1.7   1.7   1.7
4316  id_239       WD_HR    80    92    95    95    96    97    96    96    84
4317  id_239  WIND_DIREC    76    99    93    97    93    94    98    97    65
4318  id_239  WIND_SPEED   2.2   3.2   2.5   3.6     5   4.2   5.7   4.9   3.6
4319  id_239       WS_HR   1.7   2.8   2.6   3.3   3.5     5   4.9   5.2   3.6

[4320 rows x 11 columns]


# data preprocess

In [4]:
for i in range(10 ,len(train_data), 18):
    rainFall_row=train_data.iloc[i]
#     print(rainFall_row)
    for j in range(len(rainFall_row)):
        if rainFall_row[j]=='NR':
            rainFall_row[j]=0
        #print(rainFall_row[j])

In [5]:
train_y=train_data[train_data['測項'].isin(['PM2.5'])]
train_x=train_data[True^train_data['測項'].isin(['PM2.5'])]

train_x=train_x.iloc[:,3:].values
train_y=train_y.iloc[:,3:].values

train_x=np.array(train_x,dtype=np.float)
train_y=np.array(train_y,dtype=np.float)

print(train_x.shape)
print(train_y.shape)

(4080, 24)
(240, 24)


In [6]:
print(train_y.shape[0]*train_y.shape[1])

5760


In [7]:
train_x=np.reshape(train_x.transpose(),(train_x.shape[0]*train_x.shape[1]//17,17))
train_y=np.reshape(train_y, (train_y.shape[0]*train_y.shape[1],1))

In [8]:
print("train_x.shape",train_x.shape)
print("train_y.shape",train_y.shape)


train_x.shape (5760, 17)
train_y.shape (5760, 1)


# normalize

In [9]:
mean=np.mean(train_x,axis=0)
std=np.std(train_x,axis=0)
for i in range(len(train_x)):
    for j in range(len(train_x[i])):
        train_x[i][j]=(train_x[i][j]-mean[j])/std[j]

In [10]:
print(train_x)

[[-1.35579445  0.77924642  0.37595177 ... -1.30408516 -0.84222906
  -1.14132397]
 [-1.03780952  0.77924642 -0.39673931 ... -0.533142    0.09646009
   0.74086802]
 [-0.71982459  0.77924642 -0.21129345 ... -0.5014594  -0.74836014
  -0.01200878]
 ...
 [-1.35579445  0.77924642 -0.11857052 ... -0.37472902 -0.93609797
  -0.29433758]
 [-1.19680199 -0.01912771 -0.61309281 ... -1.44137641 -1.49931146
  -0.10611838]
 [-1.51478692  0.77924642  0.53048999 ... -0.56482459 -0.27901557
   0.27032002]]


# train

In [11]:
weight=np.random.randn(17,1)
learning_rate=0.000037555
for i in range(10000):
    gradient=-2*np.transpose(train_x).dot(train_y-train_x.dot(weight))
    
    weight=weight-learning_rate*gradient
    if i%500==0:
#         print(np.max(gradient))
        print("T=",i)
        print("Loss:",np.power(np.sum(np.power(train_x.dot(weight) - train_y, 2 ))/ train_x.shape[0],0.5))

T= 0
Loss: 27.298202228512686
T= 500
Loss: 27.191050553351552
T= 1000
Loss: 27.290373699897984
T= 1500
Loss: 27.42668715229437
T= 2000
Loss: 27.61319064026596
T= 2500
Loss: 27.86767703567815
T= 3000
Loss: 28.21387848569688
T= 3500
Loss: 28.68308136025182
T= 4000
Loss: 29.31595042870274
T= 4500
Loss: 30.164426342184413
T= 5000
Loss: 31.293472409467515
T= 5500
Loss: 32.782388037050914
T= 6000
Loss: 34.72544332608631
T= 6500
Loss: 37.23178077093205
T= 7000
Loss: 40.42486153487108
T= 7500
Loss: 44.44207165918599
T= 8000
Loss: 49.43524807916847
T= 8500
Loss: 55.57272469137624
T= 9000
Loss: 63.043125908820514
T= 9500
Loss: 72.06077431183924


In [12]:
##sample code
w=np.random.randn(17,1)
learning_rate=2000
adagrad_sum=np.zeros(shape=(weight.shape[0],1))
for T in range(10000):
    if(T % 500 == 0 ):
        print("T=",T)
        print("Loss:",np.power(np.sum(np.power(train_x.dot(w) - train_y, 2 ))/ train_x.shape[0],0.5))
    gradient = (-2) * np.transpose(train_x).dot(train_y-train_x.dot(w)) # 這樣為什麼算是微分??
    adagrad_sum += gradient ** 2
    w = w - learning_rate * gradient / (np.sqrt(adagrad_sum) + 0.0005)
#     w = w - learning_rate * gradient
#     print(np.max(gradient))

T= 0
Loss: 27.312389879544067
T= 500
Loss: 26.92306591288979
T= 1000
Loss: 26.922879200252662
T= 1500
Loss: 26.922738956039066
T= 2000
Loss: 26.922633510198395
T= 2500
Loss: 26.922554228400156
T= 3000
Loss: 26.922494618669297
T= 3500
Loss: 26.9224497998355
T= 4000
Loss: 26.922416101865934
T= 4500
Loss: 26.92239076535791
T= 5000
Loss: 26.922371715591048
T= 5500
Loss: 26.922357392640812
T= 6000
Loss: 26.92234662364436
T= 6500
Loss: 26.922338526760036
T= 7000
Loss: 26.922332438956936
T= 7500
Loss: 26.922327861721698
T= 8000
Loss: 26.92232442023698
T= 8500
Loss: 26.922321832688812
T= 9000
Loss: 26.92231988718974
T= 9500
Loss: 26.922318424427964


In [13]:
# 壞掉的
# weight=np.random.randn(17,1)
# learning_rate=0.000001
# adagrad_sum=np.zeros(shape=(weight.shape[0],1))
# for i in range(100):
#     gradient=-2*train_x.T.dot(train_y-train_x.dot(weight))
#     print(gradient) #不會收斂
#     print('='*20)
# #     if np.max(gradient) >1000000000000:
# #         break
#     adagrad_sum=np.power(gradient,2)*0.0000000001
#     print(adagrad_sum)
#     learning_rate=learning_rate-math.sqrt(1/(i+0.1))
#     weight=weight-learning_rate*gradient/np.sqrt(1/(i*adagrad_sum+0.00001))
# #     if i%10==0:
# #         print(weight)

# test data

In [14]:
test_data = pd.read_csv("test.csv",header=None)
r=list(range(9))
columns=['id','測項']
for i in range(len(r)):
    columns.extend(str(r[i]))
test_data.columns=columns
print(test_data)

          id          測項     0     1     2     3     4     5     6     7     8
0       id_0    AMB_TEMP    21    21    20    20    19    19    19    18    17
1       id_0         CH4   1.7   1.7   1.7   1.7   1.7   1.7   1.7   1.7   1.8
2       id_0          CO  0.39  0.36  0.36   0.4  0.53  0.55  0.34  0.31  0.23
3       id_0        NMHC  0.16  0.24  0.22  0.27  0.27  0.26  0.27  0.29   0.1
4       id_0          NO   1.3   1.3   1.3   1.3   1.4   1.6   1.2   1.1   0.9
...      ...         ...   ...   ...   ...   ...   ...   ...   ...   ...   ...
4315  id_239         THC   1.8   1.8   1.8   1.8   1.7   1.7   1.7   1.7   1.7
4316  id_239       WD_HR    80    92    95    95    96    97    96    96    84
4317  id_239  WIND_DIREC    76    99    93    97    93    94    98    97    65
4318  id_239  WIND_SPEED   2.2   3.2   2.5   3.6     5   4.2   5.7   4.9   3.6
4319  id_239       WS_HR   1.7   2.8   2.6   3.3   3.5     5   4.9   5.2   3.6

[4320 rows x 11 columns]


In [15]:
for i in range(10 ,len(test_data), 18):
    rainFall_row=test_data.iloc[i]
#     print(rainFall_row)
    for j in range(len(rainFall_row)):
        if rainFall_row[j]=='NR':
            rainFall_row[j]=0
#         print(rainFall_row[j])

In [16]:
print(test_data)

          id          測項     0     1     2     3     4     5     6     7     8
0       id_0    AMB_TEMP    21    21    20    20    19    19    19    18    17
1       id_0         CH4   1.7   1.7   1.7   1.7   1.7   1.7   1.7   1.7   1.8
2       id_0          CO  0.39  0.36  0.36   0.4  0.53  0.55  0.34  0.31  0.23
3       id_0        NMHC  0.16  0.24  0.22  0.27  0.27  0.26  0.27  0.29   0.1
4       id_0          NO   1.3   1.3   1.3   1.3   1.4   1.6   1.2   1.1   0.9
...      ...         ...   ...   ...   ...   ...   ...   ...   ...   ...   ...
4315  id_239         THC   1.8   1.8   1.8   1.8   1.7   1.7   1.7   1.7   1.7
4316  id_239       WD_HR    80    92    95    95    96    97    96    96    84
4317  id_239  WIND_DIREC    76    99    93    97    93    94    98    97    65
4318  id_239  WIND_SPEED   2.2   3.2   2.5   3.6     5   4.2   5.7   4.9   3.6
4319  id_239       WS_HR   1.7   2.8   2.6   3.3   3.5     5   4.9   5.2   3.6

[4320 rows x 11 columns]


In [17]:
test_y=test_data[test_data['測項'].isin(['PM2.5'])]
test_x=test_data[True^test_data['測項'].isin(['PM2.5'])]

test_x=test_x.iloc[:,2:].values
test_y=test_y.iloc[:,2:].values
print(test_x)

test_x=np.array(test_x,dtype=np.float)
test_y=np.array(test_y,dtype=np.float)

print(test_x.shape)
print(test_y.shape)

[['21' '21' '20' ... '19' '18' '17']
 ['1.7' '1.7' '1.7' ... '1.7' '1.7' '1.8']
 ['0.39' '0.36' '0.36' ... '0.34' '0.31' '0.23']
 ...
 ['76' '99' '93' ... '98' '97' '65']
 ['2.2' '3.2' '2.5' ... '5.7' '4.9' '3.6']
 ['1.7' '2.8' '2.6' ... '4.9' '5.2' '3.6']]
(4080, 9)
(240, 9)


In [18]:
test_x=np.reshape(test_x.transpose(),(test_x.shape[0]*test_x.shape[1]//17,17))
test_y=np.reshape(test_y, (test_y.shape[0]*test_y.shape[1],1))

In [19]:
print("X shape",test_x.shape)
print("Y shape",test_y.shape)

X shape (2160, 17)
Y shape (2160, 1)


# predict

In [20]:
sample_code_result=test_x.dot(w)
result=test_x.dot(weight)

In [21]:
f = open("my_output.csv","w")
wrt = csv.writer(f)
title = ['id','value']
wrt.writerow(title) 
for i in range(240):
    content = ['id_'+str(i),result[i][0]]
    wrt.writerow(content) 
f.close()

In [22]:
result

array([[1587.07978409],
       [1555.46437723],
       [4288.56960455],
       ...,
       [3820.08134262],
       [4241.81881288],
       [ 849.00022188]])

In [23]:
sample_code_result

array([[ -23.65979868],
       [ -66.97778079],
       [-388.33569719],
       ...,
       [-381.81882511],
       [-363.78676778],
       [-103.47510038]])